# Pêche morutière française 1500-1950 – Extraction des données

Données issues de :

Hersart de La Villemarqué Jacqueline, La  pêche morutière française de 1500 à 1950 : statistiques, climat et société, Plouzané, IFREMER, 1995, https://archimer.ifremer.fr/doc/00486/59783/




In [1]:
import pdfplumber
import re
import pandas as pd
import sqlite3 as sql
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Extraction des pages

* Commencer par le tableau de la page 86 (chiffre à insérer ici) correspondant à la page 86 du PDF, et terminer à la p. 132 (ici) correspondant à la p.134 du PDF.
* Manquent dans ce PDF les pp. *94 et 95*
* Noter aussi que par périodes les données sont lacunaires et que dans l'exposé l'auteur précise qu'il s'agit de valeurs minimum, la réalité étant en dessus.

### Mise en place

In [2]:
###  Ouvrir le fichier avec pdfplumber et créer un objet
file = "./data/villemarque_peche_morutiere/villemarque_peche_morutiere.pdf"
pdf = pdfplumber.open(file)
#    print(first_page.chars[0])

In [5]:
### Test d'inspection d'une page
page = pdf.pages[132]
# dir(page)
print('Numéro de la page : ', page.page_number)
# page.objects
p_extract = page.extract_text(x_tolerance=3, y_tolerance=3)
p_extract[:250]

Numéro de la page :  133


"134  ANNEXE \nI Nombre \nTonnage  Pêche en \nLieu de  Nombre \nAnnée  Port de départ  L *ux  pêche  d'hommes  total  tonnes  Références bibliographiques \n(tonneaux)  métriques \npartis \n1948  FECAMP  20  30  0  0  126161 STATPECH 1950 \n1948  SAINT MALO  8"

In [6]:
try :
    lines = p_extract.split('\n')
except Exception as e:
    print(e)
lines[5:8]

['(tonneaux)  métriques ',
 'partis ',
 '1948  FECAMP  20  30  0  0  126161 STATPECH 1950 ']

In [7]:
years = [re.search("^\d{4}\s.*",l).group() for l in lines if re.search("^\d{4}\s.*", l) ]
years[:3]

['1948  FECAMP  20  30  0  0  126161 STATPECH 1950 ',
 '1948  SAINT MALO  8  30  0  0  59071 STATPECH 1950 ',
 '1948  LA ROCHELLE  1  30  0  0  1871  STATPECH 1950 ']

In [8]:
s = [[y.split('  '), len(y.split('  '))] for y in years]
s[:3]

[[['1948', 'FECAMP', '20', '30', '0', '0', '126161 STATPECH 1950 '], 7],
 [['1948', 'SAINT MALO', '8', '30', '0', '0', '59071 STATPECH 1950 '], 7],
 [['1948', 'LA ROCHELLE', '1', '30', '0', '0', '1871', 'STATPECH 1950 '], 8]]

### Extraction

In [135]:
i = 86
result_list = []
while i < 133:
    try:
        page = pdf.pages[i]
        if i < 94:
            n_page = i
        else:
            n_page = i + 2
        # pn = page.page_number
        p_extract = page.extract_text(x_tolerance=3, y_tolerance=3)
    except Exception as e:
        print('P – p. ' + i + ' – ', e)
    try :
        lines = p_extract.split('\n')
    except Exception as e:
        print('L - p. ' + i + ' – ', e)
    years = [re.search("^\d{4}\s.*",l).group() for l in lines if re.search("^\d{4}\s.*", l) ]
    s = [[y.split('  '), len(y.split('  ')), n_page] for y in years]
    result_list += s
    i += 1

In [136]:
len(result_list)

2553

In [137]:
result_df = pd.DataFrame(result_list, columns=['valeurs','champs','n_page'])
result_df.head()

,valeurs,champs,n_page
0,"[1508, BREHAT, 1, 30, 0, 0, 0. MORANDIERE 1962 ]",7,86
1,"[1508, ROUEN, 4, 1, 0, 0, o' DARDEL 1941 ]",7,86
2,"[1510, SAINT BRIEUC, 1, 30, 0, 0, 0, MORANDIER...",8,86
3,"[1510, ROUEN, 1, 1, 0, 0, 0, DARDEL 1941 ]",8,86
4,"[1512, BAYONNE, 1, 30, 0, 0, 01 MORANDIERE 1962 ]",7,86


In [138]:
result_df.tail()

,valeurs,champs,n_page
2548,"[1950, | FECAMP, 9, 30, 0, 0, 19070' STATPECH ...",7,134
2549,"[1950, i FECAMP, 1, 60, 0, 0., 85 i STATPECH 1...",7,134
2550,"[1950, SAINT MALO, 13, 30, 0, 0, 3650! STATPEC...",7,134
2551,"[1950, LA ROCHELLE, 2, 30, 0, 0, 2438, STATPEC...",8,134
2552,"[1950, ; BORDEAUX, 7, 30, 0, 0,, 16407, STATPE...",8,134


### Extraction et nettoyage des ports

In [139]:
[v[1] for v in result_df['valeurs'] ][:5]

['BREHAT', 'ROUEN', 'SAINT BRIEUC', 'ROUEN', 'BAYONNE']

In [140]:
### Ajouter une colonne au dataframe contenant les ports
result_df['port'] = result_df['valeurs'].apply(lambda x: x[1])

In [141]:
result_df['port']

0             BREHAT
1              ROUEN
2       SAINT BRIEUC
3              ROUEN
4            BAYONNE
            ...     
2548        | FECAMP
2549        i FECAMP
2550      SAINT MALO
2551     LA ROCHELLE
2552      ; BORDEAUX
Name: port, Length: 2553, dtype: object

In [142]:
result_df['port'] = result_df['port'].apply(lambda x: x.replace('ST MALO', 'SAINT MALO'))
result_df['port'] = result_df['port'].apply(lambda x: x.replace('STMALO', 'SAINT MALO'))
result_df['port'] = result_df['port'].apply(lambda x: x.replace('.SAINT MALO', 'SAINT MALO'))

result_df['port'] = result_df['port'].apply(lambda x: x.replace('SANT NAZARE', 'SAINT NAZARE'))

result_df['port'] = result_df['port'].apply(lambda x: x.replace('ST BRIEUC', 'SAINT BRIEUC'))
result_df['port'] = result_df['port'].apply(lambda x: x.replace('STBRIEUC', 'SAINT BRIEUC'))
result_df['port'] = result_df['port'].apply(lambda x: x.replace('SYBRIEUC', 'SAINT BRIEUC'))
result_df['port'] = result_df['port'].apply(lambda x: x.replace('SAINT 8RIEUC', 'SAINT BRIEUC'))


result_df['port'] = result_df['port'].apply(lambda x: x.replace('inconnu', 'Inconnu'))

result_df['port'] = result_df['port'].apply(lambda x: x.replace('ST PIERRE', 'SAINT PIERRE'))

result_df['port'] = result_df['port'].apply(lambda x: x.replace('GRAVEUNES', 'GRAVELINES'))

result_df['port'] = result_df['port'].apply(lambda x: x.replace('STVALCAUX', 'SAINT-VALERY-EN-CAUX'))
result_df['port'] = result_df['port'].apply(lambda x: x.replace('ST VALERY CAUX', 'SAINT-VALERY-EN-CAUX'))
result_df['port'] = result_df['port'].apply(lambda x: x.replace('ST VAL CAUX', 'SAINT-VALERY-EN-CAUX'))

result_df['port'] = result_df['port'].apply(lambda x: x.replace('ST JEAN LUZ', 'SAINT JEAN LUZ'))
result_df['port'] = result_df['port'].apply(lambda x: x.replace('ST JEAN LUZ', 'SAINT JEAN LUZ'))

result_df['port'] = result_df['port'].apply(lambda x: x.replace("SABLES DOLONNE", "SABLES D'OLONNE"))
result_df['port'] = result_df['port'].apply(lambda x: x.replace("SABLES D'OLONNË", "SABLES D'OLONNE"))
result_df['port'] = result_df['port'].apply(lambda x: x.replace('DOLONNE', "D'OLONNE"))
result_df['port'] = result_df['port'].apply(lambda x: x.replace('SABLES OLONNE', "SABLES D'OLONNE"))

result_df['port'] = result_df['port'].apply(lambda x: x.replace('i', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace('!', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace(',', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace('.', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace(';', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace(",", '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace('j', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace('l', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace('1', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace('|', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace(':', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace('"', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace('^', '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace("' ", '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace("•", '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace("••", '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace("(", '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace("[", '').strip())
result_df['port'] = result_df['port'].apply(lambda x: x.replace("••", '').strip())

In [143]:
# dir(result_df['port'].value_counts())
t = [[l,v] for l,v in result_df['port'].value_counts().iteritems()]
t[60:65]
# t

[['ST MAL0', 1],
 ['SAINT-PIERRE', 1],
 ['ROCHEFORT', 1],
 ['SETE', 1],
 ['ST NAZAIRE', 1]]

### Extraction et nettoyage des années

In [144]:
### Ajouter une colonne au dataframe contenant les annees
result_df['annee'] = result_df['valeurs'].apply(lambda x: x[0])

In [145]:
result_df['annee']

0       1508
1       1508
2       1510
3       1510
4       1512
        ... 
2548    1950
2549    1950
2550    1950
2551    1950
2552    1950
Name: annee, Length: 2553, dtype: object

In [146]:
result_df['annee'] = result_df['annee'].apply(lambda x: x.replace("'", '').strip())
result_df['annee'] = result_df['annee'].apply(lambda x: x.replace('"', '').strip())
result_df['annee'] = result_df['annee'].apply(lambda x: x.replace('7791', '1791').strip())
result_df['annee'] = result_df['annee'].apply(lambda x: int(x))

In [147]:
count_annees = [[l,v] for l,v in result_df['annee'].value_counts().iteritems()]
count_annees[:10]

[[1786, 27],
 [1919, 26],
 [1784, 24],
 [1818, 24],
 [1899, 23],
 [1889, 23],
 [1773, 23],
 [1921, 22],
 [1932, 22],
 [1896, 21]]

### Extraction et nettoyage des effectifs des bateaux

In [148]:
### Ajouter une colonne au dataframe contenant les effectifs des bateaux partis
result_df['eff_bateaux'] = result_df['valeurs'].apply(lambda x: x[2])

In [149]:
result_df['eff_bateaux']

0        1
1        4
2        1
3        1
4        1
        ..
2548     9
2549     1
2550    13
2551     2
2552     7
Name: eff_bateaux, Length: 2553, dtype: object

In [150]:
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace("|", '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace("561", '56').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace("L", '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace(".", '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace("_", '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace("P", '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace("I", '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace("j", '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace("l", '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace('"', '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace('i', '1').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace('H', '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace('!', '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace(':', '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace(';', '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace(']', '').strip())
result_df['eff_bateaux'] = result_df['eff_bateaux'].apply(lambda x: x.replace(',', '').strip())

In [151]:
eff_bateaux = [[l,v] for l,v in result_df['eff_bateaux'].value_counts().iteritems()]
eff_bateaux[60:65]

[['o', 7], ['48', 7], ['55', 7], ['58', 6], ['80', 6]]

### Lieux de pêche

In [152]:
### Ajouter une colonne au dataframe contenant les codes des lieux de pêche
result_df['lieu_peche'] = result_df['valeurs'].apply(lambda x: x[3])

In [153]:
result_df['lieu_peche']

0       30
1        1
2       30
3        1
4       30
        ..
2548    30
2549    60
2550    30
2551    30
2552    30
Name: lieu_peche, Length: 2553, dtype: object

In [154]:
result_df['lieu_peche'] = result_df['lieu_peche'].apply(lambda x: x.replace("i", '').strip())
result_df['lieu_peche'] = result_df['lieu_peche'].apply(lambda x: x.replace("!", '').strip())
result_df['lieu_peche'] = result_df['lieu_peche'].apply(lambda x: x.replace("J", '').strip())
result_df['lieu_peche'] = result_df['lieu_peche'].apply(lambda x: x.replace("l", '').strip())
result_df['lieu_peche'] = result_df['lieu_peche'].apply(lambda x: x.replace(",", '').strip())
result_df['lieu_peche'] = result_df['lieu_peche'].apply(lambda x: x.replace("|", '').strip())
result_df['lieu_peche'] = result_df['lieu_peche'].apply(lambda x: x.replace(":", '').strip())
result_df['lieu_peche'] = result_df['lieu_peche'].apply(lambda x: x.replace("'", '').strip())
result_df['lieu_peche'] = result_df['lieu_peche'].apply(lambda x: x.replace('"', '').strip())
result_df['lieu_peche'] = result_df['lieu_peche'].apply(lambda x: x.replace('501', '50').strip())

In [155]:
lieu_peche = [[l,v] for l,v in result_df['lieu_peche'].value_counts().iteritems()]
lieu_peche[-5:]

[['75', 1], ['65 ^', 1], ['78', 1], ['17', 1], ['50 r', 1]]

### Ajout des labels des lieux de pêche

In [156]:
chemin = 'data/villemarque_peche_morutiere/villemarque_codes_lieux_peche.csv'
codes = pd.read_csv(chemin, sep='|', names=['code', 'label'])
codes

,code,label
0,1,"Lieu de pêche imprécis, Terre-Neuve au sens large"
1,10,Grand Banc de Terre-Neuve
2,20,Iles Saint-Pierre-et-Miquelon
3,30,Côtes de Terre-Neuve
4,31,Terre-Neuve : Plaisance
5,32,Terre-Neuve : Petit-Nord
6,33,Terre-Neuve : côte occidentale
7,40,Canada
8,41,Côte du Labrador
9,42,Ile Saint-Jean (Ile du Prince-Edouard)


In [157]:
codes['code'] = codes['code'].apply(lambda x: str(x))
#result_df['code'] = result_df['code'].apply(lambda x: int(x))

### Jointure des deux tableaux

In [184]:
result_dfm = result_df.merge(codes, left_on = 'lieu_peche', right_on ='code', how='left', indicator=True)
result_dfm.iloc[140:145]

,valeurs,champs,n_page,port,annee,eff_bateaux,lieu_peche,code,label,_merge
140,"[1614, HONFLEUR, 15, 30, 0, 140, 0, BREARD 1899 ]",8,88,HONFLEUR,1614,15,30,30,Côtes de Terre-Neuve,both
141,"[1614, DUNKERQUE, 7, 50, 0, 0, 0! RIN 1936 ]",7,88,DUNKERQUE,1614,7,50,50,Islande,both
142,"[1615, HONFLEUR, 10, 1, 0, 290, OTBRE ARD, 1899 ]",8,88,HONFLEUR,1615,10,1,1,"Lieu de pêche imprécis, Terre-Neuve au sens large",both
143,"[1615, SAINT MALO, 7, 32, 0, 0, 0, MORAND, JRE...",8,88,SAINT MALO,1615,7,32,32,Terre-Neuve : Petit-Nord,both
144,"[1616, HONFLEUR, ""Ï3, 1, 0, 0, "" 0 BREARD J899 ]",7,88,HONFLEUR,1616,Ï3,1,1,"Lieu de pêche imprécis, Terre-Neuve au sens large",both


## Stockage du résultat dans une BD SQLite

In [185]:
### Test
'///'.join(result_dfm.iloc[140][0]).split('///')

['1614', 'HONFLEUR', '15', '30', '0', '140', '0', 'BREARD 1899 ']

In [186]:
result_dfm['valeurs'] = result_dfm['valeurs'].apply(lambda x: '///'.join(x))
result_dfm.tail()

,valeurs,champs,n_page,port,annee,eff_bateaux,lieu_peche,code,label,_merge
2548,1950///| FECAMP///9///30///0///0///19070' STAT...,7,134,FECAMP,1950,9,30,30,Côtes de Terre-Neuve,both
2549,1950///i FECAMP///1///60///0///0.///85 i STATP...,7,134,FECAMP,1950,1,60,60,Groenland,both
2550,1950///SAINT MALO///13///30///0///0///3650! ST...,7,134,SAINT MALO,1950,13,30,30,Côtes de Terre-Neuve,both
2551,1950///LA ROCHELLE///2///30///0///0///2438///S...,8,134,LA ROCHELLE,1950,2,30,30,Côtes de Terre-Neuve,both
2552,"1950///; BORDEAUX///7///30///0///0,///16407///...",8,134,BORDEAUX,1950,7,30,30,Côtes de Terre-Neuve,both


In [187]:
db_path = 'data/peche_atlantique.sqlite'
cn = sql.connect(db_path)

In [ ]:
try:
    # ATTENTION #  
    # Ne plus exécuter après modification manuelle des valeurs dans la table #
    ### result_dfm.to_sql('peche_morutiere', cn, if_exists='replace',
           index_label='id')
except Exception as e:
    print(e)

In [189]:
cn.close()